# Self-Driving Car Engineer Nanodegree
## Project: **Finding Lane Lines on the Road**

**Import Packages**

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import math
%matplotlib inline

**LaneFinder**

class representing the pipeline. The static function *findLanes* is execution step by step:
* convertToGrayScale
* applyCannyEdgeDetection
* stripViewport
* applyHoughTransformation
 * this guy contains a filtering based on angles as well as 
 * the merging to vectors
* highlightLanes

In [4]:
class LaneFinder(object):
    def __init__(self, mpimage):
        self.img = mpimage;
        self.laneImg = mpimage
        self.imgDepth = None
    
    def show(self):
        plt.imshow(self.img)
        plt.show()
        
    def convertToGrayScale(self):
        self.laneImg = cv2.cvtColor(self.img, cv2.COLOR_RGB2GRAY)
        self.imgDepth = 'gray'
        
    def stripViewport(self):
        mask = np.zeros_like(self.laneImg)
        xsize = mask.shape[1]
        ysize = mask.shape[0]
        polyPtr = np.array(
            [ 
                [[xsize / 100 * 4, ysize], [xsize / 100 * 46, ysize /10*6],
                 [xsize / 100 * 54, ysize/10*6], [xsize / 100 * 96, ysize]
                ]
            ], dtype=np.int32)
        cv2.fillPoly(mask, polyPtr, 255)
        self.laneImg = cv2.bitwise_and(self.laneImg, mask)  
        
    def applyCannyEdgeDetection(self):
        # Define a kernel size and apply Gaussian smoothing
        kernel_size = 5
        blur_gray = cv2.GaussianBlur(self.laneImg,(kernel_size, kernel_size),0)
        # Define our parameters for Canny and apply
        low_threshold = 50
        high_threshold = 150
        self.laneImg = cv2.Canny(blur_gray, low_threshold, high_threshold)
        
    def __applyAngleFiltering(self, lines):
        angle = 35
        threshold = 15
        
        lowerThresh = math.radians(angle-threshold)
        upperThresh = math.radians(angle+threshold ) 
        resultingLines = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                alpha = math.fabs(math.atan((y2-y1)/(x2-x1)))
                if(lowerThresh <= alpha and alpha <= upperThresh):
                    resultingLines.append( [x1, y1, x2, y2] )
        return np.array(resultingLines).reshape(-1,1,4)
    
    def __mapStretchesOnVector(self, lines):
        m1 = []
        m2 = []
        b1 = []
        b2 = []
        for line in lines:
            for x1, y1, x2, y2 in line:
                m = (y2-y1)/(x2-x1)
                if m<0:
                    m1.append(m)
                    b1.append(y1-m*x1)
                else:
                    m2.append(m)
                    b2.append(y1-m*x1)
        #chosen the median instead of average to be more
        #safe against spikes
        m1 = np.median(m1)
        m2 = np.median(m2)
        b1 = np.median(b1)
        b2 = np.median(b2)
        #make sure that we do not report invalid data
        lines = []
        ysize = self.img.shape[0]
        if(not np.isnan(m1)):
            lines.append( [(ysize-b1)/m1, ysize, (ysize/10*6-b1)/m1, ysize/10*6] )
        if( not np.isnan(m2)):
            lines.append([(ysize-b2)/m2, ysize, (ysize/10*6-b2)/m2, ysize/10*6])
            
        #bring it into format    
        lines = np.array(lines, dtype=np.int32).reshape(-1,1,4);
        
        return lines
        
    def applyHoughTransformation(self):
        rho = 2 # distance resolution in pixels of the Hough grid
        theta = 1 * np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 15     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 20 #minimum number of pixels making up a line
        max_line_gap = 20    # maximum gap in pixels between connectable line segments
        
        lines = cv2.HoughLinesP(self.laneImg, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
        
        lines = self.__applyAngleFiltering(lines)
        lines = self.__mapStretchesOnVector(lines)
        #copy the 
        lineSpace = np.zeros_like(self.laneImg)#np.copy(self.img)*0
        for line in lines:
            for x1,y1,x2,y2 in line:
                cv2.line(lineSpace,(x1,y1),(x2,y2),(255,0,0),8)
        self.laneImg = lineSpace

    def highlightLanes(self):
        colorLineSpace = np.dstack((self.laneImg, self.laneImg*0, self.laneImg*0)) 
        self.img = cv2.addWeighted(self.img, 0.8, colorLineSpace, 1.0, 0)                 
        
        
        
    #static create
    def createFromImage(str_path):
        img = mpimg.imread(str_path)
        return LaneFinder(img)
    
    def findLanes(image):
        #this is the pipeline
        #unfortunately i saw the P1.ipynb after I've written the complete solution
        finder = LaneFinder(image)
        finder.convertToGrayScale()
        finder.applyCannyEdgeDetection()
        finder.stripViewport()
        #contains a angle based filtering 
        #as well as a mapping from all line-stretches to a single line
        finder.applyHoughTransformation()
        finder.highlightLanes()
        return finder.img

    #declaration of statics
    createFromImage = staticmethod(createFromImage)
    findLanes = staticmethod(findLanes)

**Test Images**

Build your pipeline to work on the images in the directory "test_images"
You should make sure your pipeline works well on these images before you try the videos.

In [5]:
import os
imageDir = str(os.path.join(os.getcwd(),"test_images"))
for filename in os.listdir(imageDir):
    if filename.endswith(".jpg") and not filename.startswith("res_"):
        fqp = os.path.join(imageDir, filename)
        fqp_out = os.path.join(imageDir,"res_" + filename)
        resultingImage = LaneFinder.findLanes(mpimg.imread(fqp))
        #for convertion from RGB to BGR to allow to store as jpeg again
        resultingImage = cv2.cvtColor(resultingImage, cv2.COLOR_RGB2BGR)
        cv2.imwrite(fqp_out, resultingImage)

**Apply LaneFinder on videos**

SolidYellowLeft

In [6]:
#output path & filename
yellow_output = 'test_videos_output/solidYellowLeft.mp4'

clip1 = VideoFileClip('test_videos/solidYellowLeft.mp4')
#apply LaneFinder on videoclip
yellow_clip = clip1.fl_image(lambda x: LaneFinder.findLanes(x))
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:25<00:00, 26.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 26.6 s


In [7]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

SolidWhiteRight

In [8]:
#output path & filename
white_output = 'test_videos_output/solidWhiteRight.mp4'

clip2 = VideoFileClip('test_videos/solidWhiteRight.mp4')
#apply LaneFinder on videoclip
white_clip = clip2.fl_image(lambda x: LaneFinder.findLanes(x))
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:07<00:00, 24.81it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 8.77 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))